In [ ]:
# データセットをロード
import numpy as np

def readucr(filename):
    data = np.loadtxt(filename, delimiter="\t")
    y = data[:, 0]
    x = data[:, 1:]
    return x, y.astype(int)

root_url = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

x_train, y_train = readucr(root_url + "FordA_TRAIN.tsv")
x_test, y_test = readucr(root_url + "FordA_TEST.tsv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

In [ ]:
# モデルを構築
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [ ]:
# トレーニングと評価
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=25, # 256から減らした(永遠に終わらない)
    num_heads=3, # 4 -> 3
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 500, 1)]     0           []                               
                                                                                                  
 multi_head_attention_32 (Multi  (None, 500, 1)      526         ['input_9[0][0]',                
 HeadAttention)                                                   'input_9[0][0]']                
                                                                                                  
 dropout_72 (Dropout)           (None, 500, 1)       0           ['multi_head_attention_32[0][0]']
                                                                                                  
 layer_normalization_64 (LayerN  (None, 500, 1)      2           ['dropout_72[0][0]']       

 ambda)                                                           'tf.__operators__.add_69[0][0]']
                                                                                                  
 conv1d_70 (Conv1D)             (None, 500, 4)       8           ['tf.__operators__.add_70[0][0]']
                                                                                                  
 dropout_79 (Dropout)           (None, 500, 4)       0           ['conv1d_70[0][0]']              
                                                                                                  
 conv1d_71 (Conv1D)             (None, 500, 1)       5           ['dropout_79[0][0]']             
                                                                                                  
 layer_normalization_71 (LayerN  (None, 500, 1)      2           ['conv1d_71[0][0]']              
 ormalization)                                                                                    
          

Epoch 32/200
45/45 [==============================] - 103s 2s/step - loss: 0.3708 - sparse_categorical_accuracy: 0.8396 - val_loss: 0.4527 - val_sparse_categorical_accuracy: 0.7947
Epoch 33/200
45/45 [==============================] - 103s 2s/step - loss: 0.3662 - sparse_categorical_accuracy: 0.8462 - val_loss: 0.4505 - val_sparse_categorical_accuracy: 0.7947
Epoch 34/200
45/45 [==============================] - 1215s 28s/step - loss: 0.3609 - sparse_categorical_accuracy: 0.8476 - val_loss: 0.4480 - val_sparse_categorical_accuracy: 0.7975
Epoch 35/200
45/45 [==============================] - 102s 2s/step - loss: 0.3511 - sparse_categorical_accuracy: 0.8562 - val_loss: 0.4451 - val_sparse_categorical_accuracy: 0.8114
Epoch 36/200
45/45 [==============================] - 103s 2s/step - loss: 0.3481 - sparse_categorical_accuracy: 0.8569 - val_loss: 0.4434 - val_sparse_categorical_accuracy: 0.8003
Epoch 37/200
45/45 [==============================] - 103s 2s/step - loss: 0.3456 - sparse_ca